In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [3]:
train_set= ('./data_pre/spec_imgs/train')
test_set=('./data_pre/spec_imgs/test')
val_set= (r'./data_pre/spec_imgs/val')

In [4]:
train_datagen = image.ImageDataGenerator(rescale= 1./255)
val_datagen= image.ImageDataGenerator(rescale= 1./255)
test_datagen= image.ImageDataGenerator(rescale= 1./255)


In [5]:

train_generator = train_datagen.flow_from_directory(train_set,batch_size =512 ,class_mode = 'categorical')
test_generator = test_datagen.flow_from_directory(test_set,shuffle=True,batch_size =128 ,class_mode = 'categorical')
validation_generator = test_datagen.flow_from_directory(val_set,shuffle=True,batch_size =128 ,class_mode = 'categorical')

Found 699 images belonging to 10 classes.
Found 151 images belonging to 10 classes.
Found 149 images belonging to 10 classes.


In [6]:
x_train, y_train = next(train_generator)
x_val,y_val= next(validation_generator)
x_test, y_test = next(test_generator)

In [8]:
import pickle
import optuna
from keras.applications import InceptionV3
from keras.models import Sequential
from keras.layers import Dense, Flatten
from tensorflow.keras.layers import Activation, BatchNormalization, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.activations import swish, relu,selu


def objective(trial):
    # Define the search space for hyperparameters
    num_layers = trial.suggest_int('num_layers', 0, 3)
    layer_size = trial.suggest_int('layer_size', 100, 10000) ######## modify the range
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    use_batch_norm = trial.suggest_categorical('use_batch_norm', [True, False])
    activation = trial.suggest_categorical('activation', ['relu','swish','selu'])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)

    # Create the MLP model with hyperparameters
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the layers in the base model
    for layer in base_model.layers[:-31]: #####   modify the number of layers
        layer.trainable = False

# Create a new model
    model = Sequential()

# Add the InceptionV3 base model to the new model
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(layer_size, input_shape=(x_train.shape[1],)))
    model.add(Activation(activation))
    model.add(Dropout(dropout_rate))
    
    for _ in range(num_layers):
        model.add(Dense(layer_size))
        if use_batch_norm:
            model.add(BatchNormalization())
        model.add(Activation(activation))
        model.add(Dropout(dropout_rate))
    
    model.add(Dense(10, activation='softmax'))

    # Compile the model with hyperparameters
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model with hyperparameters
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, mode='min', min_delta=0.001, restore_best_weights=True)
    pruning_callback = optuna.integration.TFKerasPruningCallback(trial, 'val_loss')
    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=150, callbacks=[early_stopping, pruning_callback], verbose=1)
    
    # Return the validation accuracy as the objective value
    return history.history['val_accuracy'][-1]

# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000)


save_path = './opt_study/optuna_incep_spec_r1_study.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(study, f)

# Print the best hyperparameters and objective value
best_params = study.best_params
best_value = study.best_value
print('Best Hyperparameters:', best_params)
print('Best Objective Value:', best_value)


[I 2024-04-23 10:04:24,675] A new study created in memory with name: no-name-0ceac196-f372-48d1-bfd2-9e899a8cd1b6


C:\Users\alin2\AppData\Local\Temp\ipykernel_30824\2631447824.py:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
C:\Users\alin2\AppData\Local\Temp\ipykernel_30824\2631447824.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


Epoch 1/150




[W 2024-04-23 10:04:28,454] Trial 0 failed with parameters: {'num_layers': 0, 'layer_size': 1077, 'dropout_rate': 0.337542658763078, 'use_batch_norm': True, 'activation': 'swish', 'learning_rate': 2.2284287981161347e-05} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\alin2\anaconda3\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\alin2\AppData\Local\Temp\ipykernel_30824\2631447824.py", line 52, in objective
    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=150, callbacks=[early_stopping, pruning_callback], verbose=1)
  File "c:\Users\alin2\anaconda3\lib\site-packages\keras\src\utils\traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "c:\Users\alin2\anaconda3\lib\site-packages\keras\src\engine\training.py", line 1807, in fit
    tmp_logs = self.train_function(iterator)
  File "c:\Users\alin2\an

KeyboardInterrupt: 